# Sample Code for Mixed-Integer Optimization

- Import related packages

In [1]:
import numpy as np
import cma.objective_function.mixed as f_mixed
import cma.optimizer.cmaeswm_elitist as cma_e
import cma.util.sampler as sampler

- Set optimization setting

In [2]:
def cmaeswm_elitist_run(dim, f, verbose=False): 

    # discrete space for binary variables
    if f.__class__.__name__ in ["SphereInt", "EllipsoidInt"]:
        discrete_space = np.tile(np.arange(-10, 11, 1), (f.ind, 1))
    else:
        discrete_space = np.tile(np.arange(0, 2, 1), (f.bid, 1))

    # initial values of Gaussian distribution
    init_m, init_sigma = f_mixed.initial_setting_for_gaussian(f)

    # optimizer
    samp = sampler.Sampler(f, 1)
    opt = cma_e.CMAESwM_elitist(
        dim,                                    # number of dimensions
        discrete_space,                         # definition of search space for discrete variables
        samp,                                   # sampler
        m=init_m, sigma=init_sigma,             # initial distribution parameter
        min_problem=f.minimization_problem,     # True if minimization, False if maximization
        postprocess=False                        # post-process is efficient for binary and integer optimization
    )
                    
    # run
    return opt.run(samp, logger=None, verbose=verbose)

- Set the setting of design variables
    - `dim`: number of dimensions 
    - `ind`: number of integer variables for SphereInt and EllipsoidInt
    - `bid`: number of binary variables for other objective functions
    - `max_eval`: maximum number of evaluations
    - `target_eval`: target evaluation value
- Select the objective function
    - SphereOneMax: returns the sum of evaluation values on OneMax using `bid` variables and on Sphere using other variables
    - EllipsoidOneMax: returns the sum of evaluation values on OneMax using `bid` variables and on Ellipsoid using other variables
    - SphereLeadingOnes: returns the sum of evaluation values on LeadingOnes using `bid` variables and on Sphere using other variables
    - EllipsoidLeadingOnes: returns the sum of evaluation values on LeadingOnes using `bid` variables and on Ellipsoid using other variables
    - SphereInt: returns the evaluation values on Sphere after rounding `ind` variables to nearest integer (others are continuous)
    - EllipsoidInt: returns the evaluation values on EllipsoidInt after rounding `ind` variables to nearest integer (others are continuous)

In [3]:
# ------------------------------
# setting
# ------------------------------
dim = 20                # total number of dimensions
ind = dim // 2          # number of integer variables for SphereInt and EllipsoidInt
bid = dim // 2          # number of binary variables for other objective functions
max_eval = dim * 1e5    # maximum number of evaluation
target_eval = 1e-10     # target evaluation value

# ------------------------------
# select objective function
# ------------------------------
f = f_mixed.SphereOneMax(d=dim, bid=bid, ind=0, target_eval=target_eval, max_eval=max_eval)
# f = f_mixed.EllipsoidOneMax(d=dim, bid=bid, ind=0, target_eval=target_eval, max_eval=max_eval)
# f = f_mixed.SphereLeadingOnes(d=dim, bid=bid, ind=0, target_eval=target_eval, max_eval=max_eval)
# f = f_mixed.EllipsoidLeadingOnes(d=dim, bid=bid, ind=0, target_eval=target_eval, max_eval=max_eval)
# f = f_mixed.SphereInt(d=dim, bid=0, ind=ind, target_eval=target_eval, max_eval=max_eval)
# f = f_mixed.EllipsoidInt(d=dim, bid=0, ind=ind, target_eval=target_eval, max_eval=max_eval)

- Run optimization

In [4]:
# verbose: print the internal information on terminal during the optimization if verbose=True
result = cmaeswm_elitist_run(dim, f, verbose=False)

- Confirm result

In [5]:
print("Number of evaluations: {}".format(result[0]))
print("Best evaluation value: {}".format(result[1]))
print("Is the best evaluation value is better than the target evaluation value?: {}".format(result[2]))

Number of evaluations: 1348
Best evaluation value: 7.889341211168972e-11
Is the best evaluation value is better than the target evaluation value?: True
